In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly import graph_objects as go
import plotly.io as pio
import json
from plotly.offline import plot

In [25]:
data = pd.read_csv("./index_data.csv")
data['date'] = pd.to_datetime(data['date'])

#### **动量因子风格择时信号：**

##### 需要数据： 沪深300指数和中证1000指数日收盘价 
##### 信号逻辑： 取前第一日和第十一日指数收盘价计算过去十天指数收益率：
#####        (1) 沪深300指数收益率 > 中证1000指数收益率： 当天开盘买入沪深300指数,收盘平仓
#####        (2) 沪深300指数收益率 < 中证1000指数收益率： 当天开盘买入中证1000指数，收盘平仓
#####        (3) 若两者收益率均小于0：当天空仓

In [26]:

data["CSI300_Close_1"] = data['CSI300_Close'].shift()
data["CSI300_Close_11"] = data['CSI300_Close'].shift(11)
data["CSI1000_Close_1"] = data['CSI1000_Close'].shift()
data["CSI1000_Close_11"] = data['CSI1000_Close'].shift(11)
data['10_days_CSI300_rtn'] = data['CSI300_Close_1']/data['CSI300_Close_11']-1
data['10_days_CSI1000_rtn'] = data['CSI1000_Close_1']/data['CSI1000_Close_11']-1

In [27]:
def signal1_row(row):
    if (row['10_days_CSI300_rtn'] * row['10_days_CSI1000_rtn'] > 0) & (row['10_days_CSI300_rtn'] < 0):
        row['Signal1'] = 0
        row['Signal1_day_rtn'] = 1
        row['Position1'] = '空仓'
    elif row['10_days_CSI300_rtn'] > row['10_days_CSI1000_rtn']:
        row['Signal1'] = 1
        row['Signal1_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position1'] = '满仓沪深300指数'
    else: 
        row['Signal1'] = -1
        row['Signal1_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position1'] = '满仓中证1000指数'
    return row

data = data.apply(signal1_row, axis = 1)

In [28]:
# fig1 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal1_rtn'],
#                  text = data['Position1'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig1.update_layout(template='simple_white',
#                    title='<b>动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig1.show()

#### **拥挤度动量因子风格择时信号：**

##### 需要数据： 大盘价值指数PB,小盘成长指数PB，万得全A指数PB 
##### 信号逻辑： 取前第一日和第五日指数PB计算指标：
#####           R1 = (大盘价值指数PB-小盘成长指数PB)/万得全A指数PB
#####        (1) R1(t-1) > R1(t-5)： 当天开盘买入沪深300指数,收盘平仓
#####        (2) R1(t-1) < R1(t-5)： 当天开盘买入中证1000指数，收盘平仓

In [29]:
data['LCVI_PB_1'] = data['LCVI_PB'].shift()
data['LCVI_PB_5'] = data['LCVI_PB'].shift(5)
data['SCGI_PB_1'] = data['SCGI_PB'].shift()
data['SCGI_PB_5'] = data['SCGI_PB'].shift(5)
data['WASI_PB_1'] = data['WASI_PB'].shift()
data['WASI_PB_5'] = data['WASI_PB'].shift(5)

In [30]:
def signal2_row(row):
    if (row['LCVI_PB_1'] - row['SCGI_PB_1'])/row['WASI_PB_1'] > (row['LCVI_PB_5'] - row['SCGI_PB_5'])/row['WASI_PB_5']:
        row['Signal2'] = 1
        row['Signal2_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position2'] = '满仓沪深300指数'
    else: 
        row['Signal2'] = -1
        row['Signal2_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position2'] = '满仓中证1000指数'
    return row

data = data.apply(signal2_row, axis = 1)

In [31]:
# fig2 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal2_rtn'],
#                  text = data['Position2'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig2.update_layout(template='simple_white',
#                    title='<b>拥挤度动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081",
#                    ),
#                    width=2000, 
#                    height=500)
# fig2.show()

#### **相对强弱动量因子风格择时信号（大盘价值指数和小盘成长指数）**

##### 需要数据： 大盘价值指数、小盘成长指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 大盘价值指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 小盘成长指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [32]:
data['LCVI_Close_1'] = data['LCVI_Close'].shift()
data['LCVI_Close_2'] = data['LCVI_Close'].shift(2)
data['SCGI_Close_1'] = data['SCGI_Close'].shift()
data['SCGI_Close_2'] = data['SCGI_Close'].shift(2)
data['WASI_Close_1'] = data['WASI_Close'].shift()
data['WASI_Close_2'] = data['WASI_Close'].shift(2)
data['LCVI_rtn_1'] = data['LCVI_Close_1']/data['LCVI_Close_2']
data['SCGI_rtn_1'] = data['SCGI_Close_1']/data['SCGI_Close_2']
data['WASI_rtn_1'] = data['WASI_Close_1']/data['WASI_Close_2']

In [33]:
def signal3_row(row):
    if (row['LCVI_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['SCGI_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal3'] = 0
        row['Signal3_day_rtn'] = 1
        row['Position3'] = '空仓'
    elif row['LCVI_rtn_1'] > row['SCGI_rtn_1']:
        row['Signal3'] = 1
        row['Signal3_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position3'] = '满仓沪深300指数'
    else: 
        row['Signal3'] = -1
        row['Signal3_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position3'] = '满仓中证1000指数'
    return row

data = data.apply(signal3_row, axis = 1)

In [34]:
# fig3 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal3_rtn'],
#                  text = data['Position3'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig3.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(大盘价值指数和小盘成长指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig3.show()

#### **相对强弱动量因子风格择时信号（沪深300指数和中证1000指数）**

##### 需要数据： 沪深300指数、中证1000指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 沪深300指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 中证1000指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [35]:
data['CSI300_Close_2'] = data['CSI300_Close'].shift(2)
data['CSI1000_Close_2'] = data['CSI1000_Close'].shift(2)
data['CSI300_rtn_1'] = data['CSI300_Close_1']/data['CSI300_Close_2']
data['CSI1000_rtn_1'] = data['CSI1000_Close_1']/data['CSI1000_Close_2']

In [36]:
def signal4_row(row):
    if (row['CSI300_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['CSI1000_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal4'] = 0
        row['Signal4_day_rtn'] = 1
        row['Position4'] = '空仓'
    elif row['CSI300_rtn_1'] > row['CSI1000_rtn_1']:
        row['Signal4'] = 1
        row['Signal4_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position4'] = '满仓沪深300指数'
    else: 
        row['Signal4'] = -1
        row['Signal4_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position4'] = '满仓中证1000指数'
    return row

data = data.apply(signal4_row, axis = 1)

In [37]:
# fig4 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal4_rtn'],
#                  text = data['Position4'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig4.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(沪深300指数和中证1000指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig4.show()

#### **多维动量复合风格择时对冲信号**

##### 信号逻辑： 根据以上四个信号计算指标
#####          c1 = 择时沪深300指数信号数量
#####          c2 = 择时中证1000指数信号数量
#####         (1) c1 - c2 > 2: 7.5%多IF + 7.5%空IM
#####         (2) 0 < c1 - c2 < 2: 5%多IF + 5%空IM
#####         (3) c1 - c2 = 0: 空仓
#####         (4) -2 < c1 - c2 < 0: 5%多IM + 5%空IF
#####         (5) c1 - c2 < -2: 7.5%多IM + 7.5%空IF

In [38]:
margin = 0.12
data['Signal_all'] = data['Signal1'] + data['Signal2'] + data['Signal3'] + data['Signal4']

In [39]:
def signal5_row(row):
    if (row['Signal_all'] > 2):
        row['Signal_all_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IF + 7.5%空IM'
    elif row['Signal_all'] > 0:
        row['Signal_all_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IF + 5%空IM'
    elif row['Signal_all'] == 0:
        row['Signal_all_day_rtn'] = 1
        row['Position5'] = '空仓'
    elif row['Signal_all'] > -2:
        row['Signal_all_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IM + 5%空IF'
    else:
        row['Signal_all_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IM + 7.5%空IF'
    return row

data = data.apply(signal5_row, axis = 1)

In [40]:
# fig5 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal5_rtn'],
#                  text = data['Position5'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig5.update_layout(template='simple_white',
#                    title='<b>多维动量复合风格择时对冲信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig5.show()

In [41]:
data = data.loc[data['date'] > '2010-01-01',:].reset_index(drop=True)

In [42]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,...,CSI300_Close_2,CSI1000_Close_2,CSI300_rtn_1,CSI1000_rtn_1,Signal4,Signal4_day_rtn,Position4,Signal_all,Signal_all_day_rtn,Position5
0,2010-01-04,3535.2290,4406.7650,4801.7380,4111.9040,2917.8525,3.3084,NaN,2.7787,4.3815,...,3558.8570,4336.6050,1.004728,1.012608,-1,1.003526,满仓中证1000指数,-4,1.009275,7.5%多IM + 7.5%空IF
1,2010-01-05,3564.0380,4462.2320,4829.1060,4158.2700,2941.4729,3.3449,NaN,2.8017,4.4299,...,3575.6840,4391.2810,0.988686,1.003526,-1,1.012587,满仓中证1000指数,-4,1.002774,7.5%多IM + 7.5%空IF
2,2010-01-06,3541.7270,4452.8680,4780.3700,4153.2210,2921.8748,3.3133,NaN,2.7692,4.3964,...,3535.2290,4406.7650,1.008149,1.012587,-1,0.997901,满仓中证1000指数,-4,1.002601,7.5%多IM + 7.5%空IF
3,2010-01-07,3471.4560,4361.1610,4673.6460,4072.9810,2860.3718,3.2496,NaN,2.7060,4.3115,...,3564.0380,4462.2320,0.993740,0.997901,-1,0.979405,满仓中证1000指数,-4,0.999529,7.5%多IM + 7.5%空IF
4,2010-01-08,3480.1300,4417.7710,4683.2360,4126.6380,2872.4092,3.2475,NaN,2.7084,4.3647,...,3541.7270,4452.8680,0.980159,0.979405,0,1.000000,空仓,-2,1.006551,7.5%多IM + 7.5%空IF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777,2025-07-24,4149.0368,6701.1200,8546.8107,5022.8314,5627.8596,1.4283,2.2899,0.9538,2.7432,...,4118.9582,6637.1035,1.000197,0.995498,1,1.007104,满仓沪深300指数,0,1.000000,空仓
3778,2025-07-25,4127.1634,6706.6098,8486.1673,5024.2351,5620.7290,1.4207,2.2915,0.9476,2.7463,...,4119.7685,6607.2200,1.007104,1.014212,-1,1.000819,满仓中证1000指数,-4,1.003807,7.5%多IM + 7.5%空IF
3779,2025-07-28,4135.8242,6729.9790,8499.9396,5026.9937,5640.9018,1.4219,2.2985,0.9468,2.7451,...,4149.0368,6701.1200,0.994728,1.000819,-1,1.003485,满仓中证1000指数,-3,1.000866,7.5%多IM + 7.5%空IF
3780,2025-07-29,4152.0247,6773.8843,8453.0316,5055.4619,5666.3109,1.4050,2.3120,0.9431,2.7610,...,4127.1634,6706.6098,1.002098,1.003485,0,1.000000,空仓,-2,1.001629,7.5%多IM + 7.5%空IF


In [43]:
data['Signal1_Value'] = np.cumprod(data['Signal1_day_rtn']) * 500000
data['Signal2_Value'] = np.cumprod(data['Signal2_day_rtn']) * 500000
data['Signal3_Value'] = np.cumprod(data['Signal3_day_rtn']) * 500000
data['Signal4_Value'] = np.cumprod(data['Signal4_day_rtn']) * 500000
data['Signal5_Value'] = np.cumprod(data['Signal_all_day_rtn']) * 500000

In [44]:
column_list = ['date','Signal1_Value','Position1','Signal2_Value','Position2','Signal3_Value','Position3','Signal4_Value','Position4','Signal5_Value','Position5']
data = data.loc[:,column_list]

In [45]:
data

,date,Signal1_Value,Position1,Signal2_Value,Position2,Signal3_Value,Position3,Signal4_Value,Position4,Signal5_Value,Position5
0,2010-01-04,5.017630e+05,满仓中证1000指数,5.017630e+05,满仓中证1000指数,5.017630e+05,满仓中证1000指数,5.017630e+05,满仓中证1000指数,5.046375e+05,7.5%多IM + 7.5%空IF
1,2010-01-05,5.080786e+05,满仓中证1000指数,5.080786e+05,满仓中证1000指数,5.080786e+05,满仓中证1000指数,5.080786e+05,满仓中证1000指数,5.060371e+05,7.5%多IM + 7.5%空IF
2,2010-01-06,5.070124e+05,满仓中证1000指数,5.070124e+05,满仓中证1000指数,5.070124e+05,满仓中证1000指数,5.070124e+05,满仓中证1000指数,5.073533e+05,7.5%多IM + 7.5%空IF
3,2010-01-07,4.965705e+05,满仓中证1000指数,4.965705e+05,满仓中证1000指数,4.965705e+05,满仓中证1000指数,4.965705e+05,满仓中证1000指数,5.071142e+05,7.5%多IM + 7.5%空IF
4,2010-01-08,5.030162e+05,满仓中证1000指数,5.030162e+05,满仓中证1000指数,4.965705e+05,空仓,4.965705e+05,空仓,5.104364e+05,7.5%多IM + 7.5%空IF
...,...,...,...,...,...,...,...,...,...,...,...
3777,2025-07-24,3.904997e+06,满仓中证1000指数,3.367039e+06,满仓中证1000指数,1.315496e+07,满仓沪深300指数,1.242742e+07,满仓沪深300指数,8.520733e+06,空仓
3778,2025-07-25,3.908196e+06,满仓中证1000指数,3.369797e+06,满仓中证1000指数,1.316573e+07,满仓中证1000指数,1.243760e+07,满仓中证1000指数,8.553171e+06,7.5%多IM + 7.5%空IF
3779,2025-07-28,3.921814e+06,满仓中证1000指数,3.381539e+06,满仓中证1000指数,1.316573e+07,空仓,1.248094e+07,满仓中证1000指数,8.560580e+06,7.5%多IM + 7.5%空IF
3780,2025-07-29,3.947399e+06,满仓中证1000指数,3.403600e+06,满仓中证1000指数,1.316573e+07,空仓,1.248094e+07,空仓,8.574527e+06,7.5%多IM + 7.5%空IF


In [46]:
data.to_csv('output.csv', index=False)